In [ ]:
# This notebook calculate the characterstic sets of two datasets and keeps their distinct domain count, then it compares the evolution of each of their data points
# -> count, coverage, average, dinctinct domain count

from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark.sql.functions import split
import pandas as pd
import matplotlib.pyplot as plt
import upsetplot

# Create a local SparkSession (seulement le master et pas les slaves)
# spark = SparkSession.builder.master("local").appName("test").getOrCreate() 

#créer une session dans le master
'''spark = SparkSession.builder \
    .master("spark://172.20.53.96:7077") \
    .appName("WDC-NouveauExemplePetitGraph") \
    .config("spark.executor.memory","28g") \
    .config("spark.driver.memory","28g") \
    .getOrCreate()'''
spark = SparkSession.builder.master("local").appName("WDC-NouveauExemplePetitGraph").getOrCreate()

spark.conf.set("spark.sql.shuffle.partitions",1000)

#fichiers de config qui permettent de se connecter au serveur de stockage s3 qui contient les fichiers de DataCommons
endpoint_url = 'https://s3.os-bird.glicid.fr/'
aws_access_key_id = 'bbd95ea3c1174caa88345404b84e458f'
aws_secret_access_key = 'eaf2a72ecf9845f583af7f3513c44f25'
hadoopConf = spark._jsc.hadoopConfiguration()
hadoopConf.set('fs.s3a.access.key', aws_access_key_id)
hadoopConf.set('fs.s3a.secret.key', aws_secret_access_key)
hadoopConf.set('fs.s3a.endpoint', endpoint_url)
hadoopConf.set('fs.s3a.path.style.access', 'true')
hadoopConf.set('fs.s3a.impl', 'org.apache.hadoop.fs.s3a.S3AFileSystem')

hadoopConf.set('spark.worker.cleanup.enabled', 'true')
hadoopConf.set('fs.s3a.committer.name', 'magic')

In [ ]:
from pyspark.sql.functions import udf
import tldextract
from pyspark.sql.functions import col, count
from pyspark.sql import functions as f

from pyspark.sql import Row
import re
from urllib.parse import urlparse

quad_motif = re.compile(r'([^\s]+)\s([^\s]+)\s(.+)\s([^\s]+)\s+\.')


def parseQ(l, parts):
    result=quad_motif.match(l)
    #print(result.groups())
    sub=result.group(1).strip()
    pred=result.group(2).strip()
    if pred=="<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>":
        pred="isa:"+result.group(3).strip()
  
    if sub.startswith("_:"):
        sub += result.group(4).strip().strip("<>")
        
    #print(hashstring)
    return Row(subject=sub,predicate=pred,hashdom=hash(sub)%parts)


# fonction qui extrait les top-level domains ou l'ip d'une string
@udf('string')
def extract_tld(url):
    tld = tldextract.extract(url)
    if tld.registered_domain:
        return tld.registered_domain
    else:
        # in case the URL is an IP
        return tld.domain

spark.udf.register("extract_tld", extract_tld)

from pyspark.sql.types import ArrayType, StringType
import random 

# selectionne aléatoirement num urls dans un array
@udf(ArrayType(StringType()))
def sample_tab(input_array, num):
    # Vérifiez si l'entrée est une liste non vide
    if input_array and isinstance(input_array, list):
        # Utilisez random.sample pour sélectionner aléatoirement 10 éléments sans remplacement
        selected_elements = random.sample(input_array, min(10, len(input_array)))
        return selected_elements
    else:
        return None  # Retournez None si l'entrée n'est pas valide

spark.udf.register("sample_tab", sample_tab)

parts = 1
# File file:/webdatacommons_data/home/nathan/petitGrapheRDFavant.txt does not exist
#spark.sparkContext.addFile("petitGrapheRDFavant.txt")
linesavant = spark.sparkContext.textFile(f"petitGrapheRDFavant.txt")

spavant=linesavant.map(lambda l: parseQ(l, parts)).toDF()
spavant.createOrReplaceTempView("Superavant")

# We compute the CSET with a list of the disctincts domains, the count of them and a sample of 10 subjects on the before and the after graph
resultavant = spark.sql(f"""
        SELECT
            pset,
            COUNT(subject) as count,
            COLLECT_SET(extract_tld(regexp_replace(subject, '^[^<>]*<(.+)>[^<>]*$', '$1'))) as distinct_domains,
            SIZE(COLLECT_SET(extract_tld(regexp_replace(subject, '^[^<>]*<(.+)>[^<>]*$', '$1')))) as num_distinct_domains,
            sample_tab(COLLECT_LIST(regexp_replace(subject, '^[^<>]*<(.+)>[^<>]*$', '$1')), 10) as urls
        FROM (
            SELECT
                subject,
                CONCAT_WS(' ', SORT_ARRAY(COLLECT_SET(predicate))) as pset
            FROM
                Superavant
            GROUP BY
                subject
        )
        GROUP BY
            pset
    """).cache()

resultavant.createOrReplaceTempView("CSET_avant")

#spark.sparkContext.addFile("petitGrapheRDFapres.txt")
linesapres = spark.sparkContext.textFile(f"petitGrapheRDFapres.txt")

spapres=linesapres.map(lambda l: parseQ(l, parts)).toDF()
spapres.createOrReplaceTempView("Superapres")

resultapres = spark.sql(f"""
        SELECT
            pset,
            COUNT(subject) as count,
            COLLECT_SET(extract_tld(regexp_replace(subject, '^[^<>]*<(.+)>[^<>]*$', '$1'))) as distinct_domains,
            SIZE(COLLECT_SET(extract_tld(regexp_replace(subject, '^[^<>]*<(.+)>[^<>]*$', '$1')))) as num_distinct_domains,
            sample_tab(COLLECT_LIST(regexp_replace(subject, '^[^<>]*<(.+)>[^<>]*$', '$1')),10) as urls
        FROM (
            SELECT
                subject,
                CONCAT_WS(' ', SORT_ARRAY(COLLECT_SET(predicate))) as pset
            FROM
                Superapres
            GROUP BY
                subject
        )
        GROUP BY
            pset
    """).cache()

resultapres.show(truncate=0)

resultapres.createOrReplaceTempView("CSET_apres")

In [ ]:
spark.sql("select * from CSET_avant").show(truncate=0)

In [ ]:
spark.sql("select * from CSET_apres").show(truncate=0)

In [ ]:
# affiche l'évolution en pourcentage du nombre d'occurence de chaque caracteristic set entre le graphe avant et le graphe après
csetCountEvolution = spark.sql("""
SELECT
    COALESCE(CSET_avant.pset, CSET_apres.pset) AS pset,
    CASE
        WHEN CSET_avant.count IS NULL THEN 0  -- Pset didn't exist in the first dataframe
        ELSE CSET_avant.count
    END AS cset_count_avant,
    CASE
        WHEN CSET_apres.count IS NULL THEN 0  -- Pset existed before and disappeared
        ELSE CSET_apres.count
    END AS cset_count_apres,
    CASE
        WHEN CSET_avant.pset IS NULL THEN "new"  -- Pset didn't exist in the first dataframe
        WHEN CSET_apres.pset IS NULL THEN -100.0    -- Pset existed before and disappeared
        ELSE (CSET_apres.count - CSET_avant.count) / CSET_avant.count * 100.0
    END AS percentage_count_evolution
FROM
    CSET_avant
FULL OUTER JOIN
    CSET_apres
ON
    CSET_avant.pset = CSET_apres.pset
""")
csetCountEvolution.show(truncate=0)

csetCountEvolution.createOrReplaceTempView("CSET_number_count")

In [ ]:
# affiche le num_distinct_domains du graphe avant et celui d'après, avec leur evolution en pourcentage,
# et cela pour chacun des cset


distinctDomainCountEvolution = spark.sql("""
    SELECT
        COALESCE(avant.pset, apres.pset) as pset,
        MAX(COALESCE(avant.num_distinct_domains, 0)) as avant_domain_count,
        MAX(COALESCE(apres.num_distinct_domains, 0)) as apres_domain_count,
        CASE
            WHEN COALESCE(MAX(avant.num_distinct_domains), 0) = 0 THEN "new"
            ELSE (MAX(COALESCE(apres.num_distinct_domains, 0)) - MAX(COALESCE(avant.num_distinct_domains, 0))) / MAX(COALESCE(avant.num_distinct_domains, 0)) * 100
        END as percentage_domain_evolution
    FROM
        CSET_avant avant
    FULL OUTER JOIN
        CSET_apres apres
    ON
        avant.pset = apres.pset
    GROUP BY
        COALESCE(avant.pset, apres.pset)
""")

# Display the result
distinctDomainCountEvolution.show(truncate=0)

distinctDomainCountEvolution.createOrReplaceTempView("CSET_distinct_domain_count")

In [ ]:
# coverage
def calculate_coverage(data, cname):
    sets=spark.sql(f"select pset,count from {data} where pset like '%{cname}%' ")
    sets = sets.withColumn("pset", split(sets["pset"], "\\|"))
    sets.createOrReplaceTempView("sets")
    top_sets= spark.sql("""
    SELECT  *
    FROM sets
    ORDER BY count DESC
    LIMIT 20
    """)
    #top_sets.show(truncate=0)
    
    exploded_data = top_sets.select("pset", "count") \
                    .selectExpr("pset", "count", "explode(pset) as predicate")
    distinct_predicate_count = exploded_data.select("predicate") \
                                       .distinct() \
                                       .count()
    #print(f"distinct predicate count {distinct_predicate_count}")
    
    querry = spark.sql("SELECT SUM(count) as count_sum FROM sets")
    #querry.show()
    totalcount = querry.first()["count_sum"]
    #print(f"total count {totalcount}")
    
    
    totalpredcount = totalcount*distinct_predicate_count
    #print(totalpredcount)
    
    querry = spark.sql("SELECT SUM(count * size(pset)) as count_sum  FROM sets")
    countused = querry.first()["count_sum"]

    structuredness = countused/totalpredcount
    #print(f"coverage {pred}: {structuredness}")
    return structuredness

def calculate_average(data, pred):
    # Danger injection SQL
    pred = pred.replace("'", "\\'")
    sets = spark.sql(f"SELECT pset, count FROM {data} WHERE pset LIKE '%{pred}%'")
    sets = sets.withColumn("pset", split(sets["pset"], " "))
    
    if sets.count() <= 1:
        count_used = sets.selectExpr("size(pset) * count as count_used").collect()[0][0]
        print(f"average {pred}: {count_used}")
        return count_used
    
    count_sum = sets.agg({"count": "sum"}).collect()[0][0]
    count_used = sets.selectExpr("sum(size(pset) * count) as count_used").collect()[0][0]

    average = count_used / count_sum
    #print(f"average {pred}: {average}")
    return average

from pyspark.sql.functions import col
def findISA(data, topn = None):
    if(topn is not None):
        param = f"limit {topn}"
    else:
        param = ""
    
    csall = spark.sql(f"select * from {data}")
    sets = csall.withColumn("pset", split(csall["pset"], " "))
    
    distinct_predicate = sets.selectExpr("explode(pset) as predicate", "count") \
                    .groupBy("predicate").agg(f.sum(f.col("count")).alias("count"))  \
                    .createOrReplaceTempView("newsets")
    #distinct_predicate=spark.sql(f"select * from newsets where predicate like '%isa:%' and count > 1 order by count desc {param}")   
    distinct_predicate=spark.sql(f"select * from newsets where predicate like '%isa:%' and count > 0 order by count desc {param}")   
    return distinct_predicate

In [ ]:
topavant = findISA("CSET_avant")
topapres = findISA("CSET_apres")

topavant.createOrReplaceTempView("toptypeavant")
topapres.createOrReplaceTempView("toptypeapres")

topavant.show(truncate=0)
topapres.show(truncate=0)

topavant_list = topavant.select('predicate').rdd.flatMap(lambda x: x).collect()
topapres_list = topapres.select('predicate').rdd.flatMap(lambda x: x).collect()

print(len(topavant_list))
print(len(topapres_list))

In [ ]:
#affiche le pourcentage d'évolution du nombre d'instance pour chacun des types
toptypeevolution = spark.sql("""
SELECT
    COALESCE(toptypeavant.predicate, toptypeapres.predicate) AS type,
    toptypeavant.count AS count_before,
    toptypeapres.count AS count_after,
    CASE
        WHEN toptypeavant.predicate IS NULL THEN "This type is only in the after graph"  -- Type is only in toptypeapres
        WHEN toptypeapres.predicate IS NULL THEN "This type is only in the before graph"  -- Type is only in toptypeavant
        ELSE (toptypeapres.count - toptypeavant.count) / toptypeavant.count * 100.0
    END AS percentage_count_evolution
FROM
    toptypeavant
FULL OUTER JOIN
    toptypeapres
ON
    toptypeavant.predicate = toptypeapres.predicate
""")
toptypeevolution.show(truncate=0)
toptypeevolution.createOrReplaceTempView("toptypeevolution")

In [ ]:
from pyspark.sql import Row
rowsavant = []
for pred in topavant_list:
    print(pred)
    rowsavant.append(Row(type=pred, average=float(calculate_average("CSET_avant", pred)), coverage=float(calculate_coverage("CSET_avant", pred))) )
    #print(rowsavant)

dfavant = spark.createDataFrame(rowsavant)
dfavant.createOrReplaceTempView("avcovavant")

rowsapres = []
for pred in topapres_list:
    print(pred)
    rowsapres.append(Row(type=pred, average=float(calculate_average("CSET_apres", pred)), coverage=float(calculate_coverage("CSET_apres", pred))) )
    #print(rowsapres)

dfapres = spark.createDataFrame(rowsapres)
dfapres.createOrReplaceTempView("avcovapres")

In [ ]:
spark.sql("select * from avcovavant").show(truncate=0)
spark.sql("select * from avcovapres").show(truncate=0)

In [ ]:
#affiche l'evolution du coverage et de l'average pour chacun des types
covavevolution = spark.sql("""
SELECT
    COALESCE(avcovavant.type, avcovapres.type) AS type,
    avcovavant.average AS average_before,
    avcovapres.average AS average_after,
    CASE
        WHEN avcovavant.type IS NULL THEN "This type is only in the after graph"  -- Type is only in avcovapres
        WHEN avcovapres.type IS NULL THEN "This type is only in the before graph"  -- Type is only in avcovavant
        ELSE (avcovapres.average - avcovavant.average) / avcovavant.average * 100.0
    END AS percentage_average_evolution,
    avcovavant.coverage AS coverage_before,
    avcovapres.coverage AS coverage_after,
    CASE
        WHEN avcovavant.type IS NULL THEN "This type is only in the after graph"  -- Type is only in avcovapres
        WHEN avcovapres.type IS NULL THEN "This type is only in the before graph"  -- Type is only in avcovavant
        ELSE (avcovapres.coverage - avcovavant.coverage) / avcovavant.coverage * 100.0
    END AS percentage_coverage_evolution
FROM
    avcovavant
FULL OUTER JOIN
    avcovapres
ON
    avcovavant.type = avcovapres.type
""")
covavevolution.show(truncate=0)
covavevolution.createOrReplaceTempView("covavevolution")

In [ ]:
combine = spark.sql("""
select * 
from covavevolution 
FULL OUTER JOIN 
    toptypeevolution
ON
    toptypeevolution.type = covavevolution.type""")
combine.show(truncate=0)